# Dual view of Python and SQL 

In [1]:
import os

import pandas as pd
from sqlalchemy import create_engine
import numpy as np

creds = """
POSTGRES_USERNAME=postgres
POSTGRES_PASSWORD=postgres
"""

with open('./.env', 'w') as f:
    f.write(creds)

postgres_connection_string = "postgres://{username}:{password}@{host}:{port}/{database}?gssencmode=disable".format(
    username="postgres",
    password="postgres",
    host="localhost",
    port="5432",
    database="lahman_baseball"
)

engine = create_engine(postgres_connection_string)

batting_sql = "SELECT * FROM batting;"

# use the connection to run a query using pandas!
batting_df = pd.read_sql(batting_sql, con=engine)
batting_df.head()

,playerid,yearid,stint,teamid,lgid,g,ab,r,h,h2b,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,NaN


In [2]:
# Clean-up the engine when done
engine.dispose()

In [3]:
# study the columns and dtypes of batting_df
batting_df.dtypes

playerid     object
yearid        int64
stint         int64
teamid       object
lgid         object
g             int64
ab            int64
r             int64
h             int64
h2b           int64
h3b           int64
hr            int64
rbi         float64
sb          float64
cs          float64
bb            int64
so          float64
ibb         float64
hbp         float64
sh          float64
sf          float64
gidp        float64
dtype: object

In [4]:
# Question 1 What range of years does the provided database cover?
# learned from prior research that batting fielding and pitching tables have the 'yearid' field
max_year=batting_df.yearid.max()
print(max_year)

2016


In [5]:
min_year=batting_df.yearid.min()
print(min_year)

1871


## Question 1 The range of years is 1871 to 2016 from the batting_df field: yearid

# For the next questions bring in more dataframes

In [6]:
appearances_sql = "SELECT * FROM appearances;"
appearances_df = pd.read_sql(appearances_sql, con=engine)

In [7]:
people_sql = "SELECT * FROM people;"
people_df = pd.read_sql(people_sql, con=engine)

In [8]:
fielding_sql = "SELECT * FROM fielding;"
fielding_df = pd.read_sql(fielding_sql, con=engine)

In [9]:
pitching_sql = "SELECT * FROM pitching;"
pitching_df = pd.read_sql(pitching_sql, con=engine)

In [10]:
teams_sql = "SELECT * FROM teams;"
teams_df = pd.read_sql(teams_sql, con=engine)

In [20]:
salaries_sql = "SELECT * FROM salaries;"
salaries_df = pd.read_sql(salaries_sql, con=engine)
schools_sql = "SELECT * FROM schools;"
schools_df = pd.read_sql(schools_sql, con=engine)
collegeplaying_sql = "SELECT * FROM collegeplaying;"
collegeplaying_df = pd.read_sql(collegeplaying_sql, con=engine)

In [11]:
# Question 2 Find the name and height of the shortest player in the database. How many games did he play in? 
# What is the name of the team for which he played?

# learned from prior research that tables: people appearances and teams hold this info
# namefirst namelast player id and height from people
# playerid teamid g_all from appearances
# teamid name from teams

In [12]:
# Get columns from tables
people_df.columns

Index(['playerid', 'birthyear', 'birthmonth', 'birthday', 'birthcountry',
       'birthstate', 'birthcity', 'deathyear', 'deathmonth', 'deathday',
       'deathcountry', 'deathstate', 'deathcity', 'namefirst', 'namelast',
       'namegiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalgame',
       'retroid', 'bbrefid'],
      dtype='object')

In [13]:
people_df.height.min()

43.0

In [14]:
shortest_player=people_df[['playerid','namefirst','namelast','height']]
shortest_player=shortest_player.sort_values(by='height')
shortest_player.head(1)

,playerid,namefirst,namelast,height
5843,gaedeed01,Eddie,Gaedel,43.0


In [15]:
#game count for shortest player
shortest_player_appearances =  appearances_df[appearances_df['playerid']=='gaedeed01']
shortest_player_appearances=shortest_player_appearances[['playerid', 'teamid', 'g_all']]
shortest_player_appearances.head()

,playerid,teamid,g_all
35173,gaedeed01,SLA,1


In [16]:
shortest_player_team =teams_df[teams_df['teamid']=='SLA']
shortest_player_team=shortest_player_team[['teamid','name']]
shortest_player_team.head(1)

,teamid,name
412,SLA,St. Louis Browns


In [17]:
question_2_df= shortest_player.merge(shortest_player_appearances,on='playerid').merge(shortest_player_team,on='teamid')
question_2_df.head(1)

,playerid,namefirst,namelast,height,teamid,g_all,name
0,gaedeed01,Eddie,Gaedel,43.0,SLA,1,St. Louis Browns


In [18]:
question_2_df.rename({'namefirst': 'first_name',
                                    'namelast':'last_name','name': 'team','g_all': 'games_played'},
                                    axis=1, inplace=True)
question_2_df.head(1)

,playerid,first_name,last_name,height,teamid,games_played,team
0,gaedeed01,Eddie,Gaedel,43.0,SLA,1,St. Louis Browns


## Question 2 Eddie Gaedel was 43" tall and played one game for the St. Louis Browns

In [ ]:
# Find all players in the database who played at Vanderbilt University. 
# Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. 
# Sort this list in descending order by the total salary earned. 
# Which Vanderbilt player earned the most money in the majors?

# Learned from prior research that tables: people, salaries, schools, collegeplaying hold this data
# .. playerid firstname, lastneame from people
#    playerid, salary from salaries
#    playerid, schoolid from collegeplaying
#    schoolid, schoolname from schools

In [26]:
schools_df.columns

Index(['schoolid', 'schoolname', 'schoolcity', 'schoolstate', 'schoolnick'], dtype='object')

In [31]:
school=schools_df[schools_df.schoolname.str.startswith('Vand')]
school=school[['schoolid','schoolname']]
school.head()

,schoolid,schoolname
1110,vandy,Vanderbilt University


In [33]:
collegeplaying_df.columns

Index(['playerid', 'schoolid', 'yearid'], dtype='object')

In [32]:
people_df.columns

Index(['playerid', 'birthyear', 'birthmonth', 'birthday', 'birthcountry',
       'birthstate', 'birthcity', 'deathyear', 'deathmonth', 'deathday',
       'deathcountry', 'deathstate', 'deathcity', 'namefirst', 'namelast',
       'namegiven', 'weight', 'height', 'bats', 'throws', 'debut', 'finalgame',
       'retroid', 'bbrefid'],
      dtype='object')

In [51]:
salaries_df.columns

Index(['yearid', 'teamid', 'lgid', 'playerid', 'salary'], dtype='object')

In [50]:
# isolate the Vanderbilt school id
player_school=collegeplaying_df[collegeplaying_df.schoolid.str.startswith('vandy')]
player_school=player_school[['playerid','schoolid']]
player_school.head(1)

,playerid,schoolid
232,alvarpe01,vandy


In [43]:
# identify the Vandy Players
print(player_school.playerid.unique())

['alvarpe01' 'baxtemi01' 'chrisni01' 'colliwi01' 'corajo01' 'embresl01'
 'flahery01' 'grayso01' 'hendrha01' 'katama01' 'lewisje01' 'madissc01'
 'minormi01' 'mooresc01' 'mossma01' 'pauljo01' 'priceda01' 'priorma01'
 'richaan01' 'sandesc01' 'sewelri01' 'sowerje01' 'willimi01' 'zeidjo01']


In [54]:
# join playerid from collegeplaying to people for namefirst and namelast filtered to the Vandy players list
# add salaries from salaries also joined on playerid = need to sum salaries and have unique rocords one per player
question_3_df= player_school.merge(school,on='schoolid').merge(people_df,on='playerid').merge(salaries_df,on='playerid')
question_3_df.head(5)

,playerid,schoolid,schoolname,birthyear,birthmonth,birthday,birthcountry,birthstate,birthcity,deathyear,...,bats,throws,debut,finalgame,retroid,bbrefid,yearid,teamid,lgid,salary
0,alvarpe01,vandy,Vanderbilt University,1987.0,2.0,6.0,D.R.,Distrito Nacional,Santo Domingo,NaN,...,L,R,2010-06-16,2016-10-01,alvap001,alvarpe01,2011,PIT,NL,2050000.0
1,alvarpe01,vandy,Vanderbilt University,1987.0,2.0,6.0,D.R.,Distrito Nacional,Santo Domingo,NaN,...,L,R,2010-06-16,2016-10-01,alvap001,alvarpe01,2012,PIT,NL,2200000.0
2,alvarpe01,vandy,Vanderbilt University,1987.0,2.0,6.0,D.R.,Distrito Nacional,Santo Domingo,NaN,...,L,R,2010-06-16,2016-10-01,alvap001,alvarpe01,2013,PIT,NL,700000.0
3,alvarpe01,vandy,Vanderbilt University,1987.0,2.0,6.0,D.R.,Distrito Nacional,Santo Domingo,NaN,...,L,R,2010-06-16,2016-10-01,alvap001,alvarpe01,2014,PIT,NL,4250000.0
4,alvarpe01,vandy,Vanderbilt University,1987.0,2.0,6.0,D.R.,Distrito Nacional,Santo Domingo,NaN,...,L,R,2010-06-16,2016-10-01,alvap001,alvarpe01,2015,PIT,NL,5750000.0


In [73]:
question_3_df.groupby(['schoolname','namefirst','namelast'])['salary'].sum()

question_3_df.head()

AttributeError: 'function' object has no attribute 'groupby'

In [74]:
question_3_df = question_3_df.sort_values(by='salary', ascending=False)

AttributeError: 'function' object has no attribute 'sort_values'